# Top N Recommendation via Latent Factor(s) Model using SGD(Stochastic Gradient Descent)
## 1. prepare dataset

In [21]:
import pandas as pd
# remember to convert dataset into utf-8 explicitly, eg: save as utf-8 via notebook
# http://www2.informatik.uni-freiburg.de/~cziegler/BX/

In [22]:
# import book data
bookFile='./data/BX-Books.csv'
books=pd.read_csv(bookFile,sep=";",header=0,error_bad_lines=False,usecols=[0,1,2],index_col=0,names=["isbn","title","author"])
# error_bad_lines=False will ignore rows with errors
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [23]:
# import rating data
dataFile='./data/BX-Book-Ratings.csv'
data=pd.read_csv(dataFile,sep=";",header=0,names=["user","isbn","rating"])
data.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


### ensure the isbn is valid

In [24]:
# subset data to only valid isbn in books database
data = data[data["isbn"].isin(books.index)]
data.shape

(1031175, 3)

### Setup a function to find book metadata

In [25]:
def bookMeta(isbn):
    title = books.at[isbn,"title"]
    author = books.at[isbn,"author"]
    return title, author
# testing
bookMeta("0671027360")

('Angels &amp; Demons', 'Dan Brown')

### setup a function to find the favorite books for a user

In [26]:
def favBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N]
    sortedRatings["title"]=sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings
# testing
favBooks(204622,5)

,user,isbn,rating,title
844955,204622,0967560500,10.0,"(Natural Hormonal Enhancement, Rob Faigin)"
844935,204622,0671027360,10.0,"(Angels &amp; Demons, Dan Brown)"
844926,204622,0385504209,10.0,"(The Da Vinci Code, Dan Brown)"
844958,204622,097173660X,9.0,"(Life After School Explained, Cap &amp; Compass)"
844920,204622,0060935464,9.0,"(To Kill a Mockingbird, Harper Lee)"


## 2. Creating the Rating Matrix

In [27]:
data.shape

(1031175, 3)

### deciding the size of the matrix

#### isbn size

In [28]:
usersPerISBN = data.isbn.value_counts()
usersPerISBN.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [29]:
usersPerISBN.shape

(270170,)

#### user size

In [30]:
ISBNsPerUser = data.user.value_counts()
ISBNsPerUser.head(10)

11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
212898     4290
278418     3996
76352      3329
110973     2971
235105     2943
Name: user, dtype: int64

In [31]:
ISBNsPerUser.shape

(92107,)

### reduce the sparse dataset

In [32]:
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)] # isbn is the index of usersPerISBN
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)] # user is the index of ISBNsPerUser
data.shape

(405709, 3)

### create the rating matrix

In [34]:
from scipy.sparse import coo_matrix
data['user'] = data['user'].astype("category") # creates an internal map of user ids to integer codes
data['isbn'] = data['isbn'].astype("category") # creates an internal map of isbns to integer codes

R = coo_matrix((data['rating'].astype(float),
               (data['user'].cat.codes.copy(),
                data['isbn'].cat.codes.copy())))
print("the shape of R matrix is: ", R.shape)
print("the length of R matrix is: ", len(R.data))

the shape of R matrix is:  (10706, 15451)
the length of R matrix is:  405709


In [36]:
# coo_matrix can greatly reduce the length(memory usage) of the matrix by not storing NaN values, but you can still locate it
# First element value:
print(R.data[0])
# row number of the first element:
print(R.row[0])
# col number of the first element：
print(R.col[0])


0.0
10633
3053


## 3. Initialize factor matrices

In [38]:
# dimension of the matrix & Factors amount(K)
M,N = R.shape
K = 3

In [39]:
import numpy as np
P = np.random.rand(M,K)
Q = np.random.rand(K,N)

## 4. Compute the error

In [40]:
from numpy.linalg import norm

# setup a function to measure RMSE
def error(R,P,Q,lamda=0.02): # lamda is the regularization factor, to penalize models 
    ratings = R.data
    rows = R.row # shortcut for R.row[some_position_of_coo_matrix]
    cols = R.col # shortcut for R.col[some_position_of_coo_matrix]
    e = 0 
    for ui in range(len(ratings)): # iterate every rating
        rui=ratings[ui] # is the rating iterated to
        u = rows[ui] # return row position of the sequence of the rating in rating table
        i = cols[ui] # return row position of the sequence of the rating in rating table
        if rui>0:
            e= e + pow(rui-np.dot(P[u,:],Q[:,i]),2)+\
                lamda*(pow(norm(P[u,:]),2)+pow(norm(Q[:,i]),2))
    return e
# testing
error(R,P,Q)

7169596.729739941

In [42]:
# RMSE - Root Mean Sqaured Error
rmse = np.sqrt(error(R,P,Q)/len(R.data)) # measure of the error per rating
rmse

4.20378061583855

## 5. Update the factor matrices - minimize the error using SGD

In [66]:
# look at the error for a rating a time
def SGD(R, K, lamda=0.02,steps=10, gamma=0.001):
    
    M,N = R.shape
    P = np.random.rand(M,K)
    Q = np.random.rand(K,N)
    
    rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
    print("Initial RMSE: "+str(rmse))
    
    for step in xrange(steps):
        for ui in range(len(R.data)):
            rui=R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            if rui>0:
                eui=rui-np.dot(P[u,:],Q[:,i]) # calculate single error of a rating
                P[u,:]=P[u,:]+gamma*2*(eui*Q[:,i]-lamda*P[u,:]) # calculate the slope and takea step in the downward direction of the slope
                Q[:,i]=Q[:,i]+gamma*2*(eui*P[u,:]-lamda*Q[:,i]) # calculate the slope and takea step in the downward direction of the slope
        rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
        if rmse<0.5:
            break
    print("Final RMSE: "+str(rmse))
    return P,Q

,user,isbn,rating,title
845417,204813,0399149848,10.0,"(Birthright, Nora Roberts)"
845407,204813,0385504209,10.0,"(The Da Vinci Code, Dan Brown)"
845382,204813,0373218036,10.0,"(Truly, Madly Manhattan, Nora Roberts)"
845359,204813,0142001805,10.0,"(The Eyre Affair: A Novel, Jasper Fforde)"
845431,204813,0446527793,10.0,"(The Guardian, Nicholas Sparks)"
845416,204813,0399149392,10.0,"(Chesapeake Blue (Quinn Brothers (Hardcover)),..."
845432,204813,0446531332,9.0,"(Nights in Rodanthe, Nicholas Sparks)"
845434,204813,0446606243,9.0,"(The Tenth Justice, Brad Meltzer)"
845451,204813,0671027360,9.0,"(Angels &amp; Demons, Dan Brown)"
845433,204813,0446532452,9.0,"(The Wedding, Nicholas Sparks)"


In [67]:
(P,Q)=SGD(R,K=2,gamma=0.0007,lamda=0.01, steps=100)

C:\Users\leolin\Anaconda3\lib\site-packages\pandas\core\apply.py:251: RuntimeWarning: Mean of empty slice
  labels=labels)


0    (Waiting For Nick (Silhouette Special Edition)...
1           (Wringer (Trophy Newbery), Jerry Spinelli)
2    (The Star Wars Trilogy: Star Wars, the Empire ...
3          (One, Two, Buckle My Shoe, Agatha Christie)
4                          (On the Road, Jack Kerouac)
5                 (Dead Poets Society, N.H. Kleinbaum)
6     (Go Ask Alice (Avon/Flare Book), James Jennings)
7                        (Carolina Moon, Nora Roberts)
8    (Illusions: The Adventures of a Reluctant Mess...
9    (You Just Don't Duct Tape a Baby!: True Tales ...
Name: isbn, dtype: object